In [1]:
import numpy as np
x = np.random.normal(0, 1, 100_000_000)

In [4]:
def py_count(array, threshold):
    counter = 0
    for entry in array:
        if entry > threshold:
            counter += 1
    
    return counter

In [8]:
%timeit py_count(x[:10_000_000], 4)

2.18 s ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
def np_count(array, threshold):
    above_thresholds = array > threshold
    return np.count_nonzero(above_thresholds)

%timeit np_count(x[:10_000_000], 4)

5.77 ms ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
print("""

extern "C" 
int c_count(int len, double* array, double threshold)
{
  int i;
  int counter = 0;
  
  for (i = 0; i < len; ++i)
    if (array[i] > threshold)
      counter ++;

  return counter;
}
""", file=open("source.C", "w"))

!gcc source.C -o mylib.so -fPIC --shared

In [12]:
!ls

mylib.so  source.C  src.C  Untitled1.ipynb  Untitled.ipynb  validate_user.ipynb


In [14]:
import ctypes
mylib = ctypes.CDLL("./mylib.so")

mylib.c_count

In [15]:
mylib.c_count.restype = ctypes.c_int
mylib.c_count.argtypes = [
    ctypes.c_int,
    np.ctypeslib.ndpointer(np.float64),
    ctypes.c_double]


In [18]:
%timeit mylib.c_count(len(x), x, 4.)

248 ms ± 672 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit np_count(x, 4.)

78.9 ms ± 6.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
